In [43]:
import pandas as pd
from ollama import Client, AsyncClient
from ollama import ChatResponse
import json
from tqdm import tqdm
import numpy as np
from scipy.stats import kendalltau
from sklearn.metrics import ndcg_score

In [44]:
corpus = pd.read_json("corpus.jsonl", lines=True)
queries = pd.read_json("queries.jsonl", lines=True)
corpus.set_index("_id", inplace=True)
queries.set_index("_id", inplace=True)
qrels = pd.read_csv("dev.tsv", sep="\t")
qrels

,query-id,corpus-id,score
0,0,620,2
1,0,621,2
2,0,622,2
3,0,616,1
4,0,617,1
...,...,...,...
4993,557,188393,1
4994,557,188394,1
4995,558,188507,2
4996,558,188508,2


In [ ]:
"""Учитывая запрос и отрывок, необходимо указать оценку по целочисленной шкале от 0 до 3 со следующими значениями:

0 = означает, что отрывок не имеет никакого отношения к запросу,
1 = означает, что отрывок, по-видимому, связан с запросом, но не отвечает на него,
2 = означает, что отрывок содержит некоторый ответ на запрос, но ответ может быть немного неясным или скрыт среди посторонней информации, и
3 = означает, что отрывок посвящен запросу и содержит точный ответ.

Важное указание: Присвойте категорию 1, если отрывок в некоторой степени связан с темой, но не полностью, категорию 2, если отрывок содержит что-то очень важное, связанное со всей темой, но также содержит дополнительную информацию, и категорию 3, если отрывок только и полностью относится к теме. Если ни один из вышеперечисленных критериев не удовлетворяет, присвойте ему категорию 0.

Запрос: {query}
Отрывок: {passage}

Разбейте эту задачу на этапы:
Рассмотрите основную цель поиска. Оцените, насколько содержание соответствует вероятному намерению запроса (M).
Оцените, насколько достоверна информация в тексте (T).
Рассмотрите вышеперечисленные аспекты и относительную важность каждого из них и определите окончательную оценку (O). Окончательная оценка должна быть целым числом.
Не указывайте код в результатах. Укажите каждую оценку в формате: {{"score": N}}: целочисленная оценка без каких-либо обоснований."""



class OllamaJudge:
  def __init__(self, model="qwen2.5:7b-instruct"):
    self.model = model
    self.client = Client(
    # host='http://188.75.5.101:8080',
    # headers={'Authorization': 'Bearer ' + "64fd71db4c0cb8a99a1e9f8ca852a87cff7847c9bbd6e7559e2f9e9bb5114666"}
    )
  def generate_judgement(self, query, passage):
    system_prompt = f"""Вы - эксперт по оценке релевантности в информационном поиске. Ваша задача - оценить, насколько документ соответствует информационной потребности пользователя.

### ШКАЛА ОЦЕНОК
Используйте следующую шкалу (целые числа от 0 до 2):

**0 — НЕРЕЛЕВАНТНО**
- Документ не связан с темой запроса
- Не содержит полезной информации для пользователя
- Может содержать ключевые слова, но без смысловой связи

**1 — ЧАСТИЧНО РЕЛЕВАНТНО**
- Касается темы запроса, но не отвечает полностью на информационную потребность
- Содержит общую информацию, часть фактов или косвенные данные
- Может требовать от пользователя дополнительного поиска

**2 — ПОЛНОСТЬЮ РЕЛЕВАНТНО**
- Прямо и полно отвечает на запрос пользователя
- Содержит точную, конкретную и достаточную информацию
- Позволяет пользователю удовлетворить информационную потребность без дополнительного поиска

### ПРОЦЕДУРА ОЦЕНКИ
Выполните оценку в три этапа:

**ЭТАП 1: Анализ информационной потребности**
1. Определите основную тему запроса
2. Выявите скрытое намерение пользователя (что пользователь хочет узнать/сделать)

**ЭТАП 2: Оценка по пяти критериям (каждый по шкале 0-1)**
A. Тематическое соответствие: документ относится к теме запроса? (0/1)
B. Полнота ответа: документ содержит полный ответ на запрос? (0/1)  
C. Конкретность: информация точная и детализированная? (0/1)
D. Достоверность: информация кажется проверяемой и надежной? (0/1)
E. Полезность: документ помогает пользователю достичь цели? (0/1)

**ЭТАП 3: Итоговая оценка**
- Если сумма критериев < 2 → ОЦЕНКА 0
- Если сумма критериев = 2-3 → ОЦЕНКА 1  
- Если сумма критериев = 4-5 → ОЦЕНКА 2

### ВАЖНЫЕ ПРАВИЛА
1. Будьте консервативны при назначении оценки 2
2. Оценка 0 назначается только при полном отсутствии тематической связи
3. При неопределенности склоняйтесь к более низкой оценке
4. Не учитывайте качество написания, только информационную релевантность

### ФОРМАТ ОТВЕТА
Выведите ТОЛЬКО JSON в следующем формате:
{{"final_score": N}}
Где N — целое число 0, 1 или 2.
"""
    prompt = f"""Дан поисковый запрос и текст. Необходимо присвоить оценку по шкале 0–2, оценивая релевантность текста запросу:
0 — текст не связан с запросом и не помогает пользователю получить нужную информацию по теме запроса.
1 — текст относится к теме запроса и может помочь пользователю понять или искать ответ, но не раскрывает информационную потребность полностью (содержит только часть информации, общие сведения или косвенные факты).
2 — текст является полностью релевантным информационной потребности запроса: содержит точную информацию, соответствующую запросу, и может считаться ответом на запрос или прямым источником нужных данных.

Ниже приведены некоторые примеры категоризации релевантности:
Запрос: Памятник на месте рождения Пушкина стоит не там.
Текст: Второй сезон был позитивно оценён критиками. Примечания. Памятник Пушкину на Бауманской улице в Москве — скульптурное изображение (бюст), установленное на месте предполагаемого рождения русского поэта и писателя Александра Сергеевича Пушкина.
Шаги: 
1. Определи информационную потребность запроса.
- Цель - получить подтверждение или опровержение того, что памятник Пушкину стоит не там.
2. Оцени, насколько текст соответствует этой потребности (M).
- В тексте упоминается памятник Пушкину, есть его описание, но нет каких либо подтверждений или опровержений, что но стоит не там.
- Текст частично покрывает информационную потребность запроса, но не дает главного ответ.
{{"score": 1}}

Запрос:  Японский автомобиль (на илл.) побывал в роли скейтборда, танцевал брейк-данс и сыграл в пейнтбол.
Текст:  // CarSales : сайт. — 2008.. Роликовая доска или скейтборд (жарг. скейт, от англ. skateboard — «роликовая доска») — доска, состоящая обычно из 7 слоёв канадского шпона, установленная на колёса небольшого диаметра (ролики).
1. Определи информационную потребность запроса.
- Цель - получить подтверждение или опровержение того, что японский автомобиль побывал в роли скейтборда, танцевал брейк-данс и сыграл в пейнтбол.
2. Оцени, насколько текст соответствует этой потребности (M).
- В тексте подробно описывается скейтборда, но нет ни слова о японской машине, побывавшей в роли скейтборда.
3. Определи итоговую оценку (O).
- Текст частично покрывает информационную потребность запроса, но не дает главного ответ.
{{"score": 1}}

Важные правила релевантности:
1 присваивается, если текст относится к той же теме или объекту, даже если он не даёт ответа или даёт только часть факта.
2 присваивается только если текст полностью покрывает информационную потребность, а не только часть.
0 присваивается только когда отсутствует связь с темой или объектом запроса.
Запрос: {query}
Текст: {passage}
Разбей оценку на шаги:
Определи информационную потребность запроса.
Оцени, насколько текст соответствует этой потребности (M).
Определи итоговую оценку (O). Итог должен быть целым числом 0–2.
Не пиши объяснений. Выведи только результат строго в формате (где N число):
{{"score": N}}
"""
    response: ChatResponse = self.client.chat(model=self.model, messages=[
      # {
      #   'role': 'system',
      #   'content': system_prompt
      # },
      {
        'role': 'user',
        'content': prompt,
      },
], format={
  "type": "object",
    "properties": {
      "score": {
        "type": "integer"
      },
    },
    "required": [
      "score",
    ]}, options={
      "num_predict": 32
    }, think=False)
    return response.message.content
    

In [46]:
judge = OllamaJudge("gemma3:4b")

In [47]:
print("Query: " + queries.loc[163]["text"] + '\n' + "Passage: " + corpus.loc[158633]["text"])

Query: Визуальной новелле удалось получить максимальный балл на Metacritic.
Passage: Он неоднократно приезжал в Веймарскую школу, чтобы посмотреть, как себя ведут советские студенты и не нарушают ли они режим. Перед стажировкой в Веймаре он ещё два месяца проучился в Лейпцигском университете, осваивая немецкий язык, а по возвращении в МАрхИ прошёл краткосрочные курсы в Институте иностранных языков имени Мориса Тореза, где успешно сдал государственный экзамен на специальность военного переводчика (военную лексику ему преподавал старший лейтенант). В ходе обучения в Инязе он прошёл восемь семестров немецкого.


In [48]:
judge.generate_judgement(queries.loc[163]["text"], corpus.loc[158633]["text"])

<coroutine object OllamaJudge.generate_judgement at 0x1194b2640>

In [49]:
def create_balanced_subset(df,all_corpus_ids, num_queries=20, samples_per_score=2, random_state=42):
    np.random.seed(random_state)
    
    all_query_ids = df['query-id'].unique()
    selected_queries = np.random.choice(
        all_query_ids, 
        min(num_queries, len(all_query_ids)), 
        replace=False
    )
    
    samples = []
    
    for query_id in selected_queries:
        query_data = df[df['query-id'] == query_id]
        
        positives_by_score = {}
        for score in [1, 2]:
            score_data = query_data[query_data['score'] == score]
            if len(score_data) > 0:
                positives_by_score[score] = score_data
        
        for score in [1, 2]:
            if score in positives_by_score:
                score_data = positives_by_score[score]
                if len(score_data) > samples_per_score:
                    selected = score_data.sample(samples_per_score, random_state=random_state)
                else:
                    selected = score_data
                samples.append(selected)
        
        used_corpus_ids = set(query_data['corpus-id'])
        available_negatives = list(all_corpus_ids - used_corpus_ids)
        
        if available_negatives and samples_per_score > 0:
            num_negatives = min(samples_per_score, len(available_negatives))
            negative_corpus_ids = np.random.choice(available_negatives, num_negatives, replace=False)
            
            for corpus_id in negative_corpus_ids:
                samples.append(pd.DataFrame([{
                    'query-id': query_id,
                    'corpus-id': corpus_id,
                    'score': 0
                }]))
    
    if samples:
        result_df = pd.concat(samples, ignore_index=True)
    else:
        result_df = pd.DataFrame(columns=['query-id', 'corpus-id', 'score'])
    
    return result_df
# test = create_balanced_subset(qrels, set(corpus.index.to_list()), num_queries=30, samples_per_score=4)

In [ ]:
results = []
test = qrels.head(100)
for idx, row in tqdm(qrels.iterrows(), total=len(qrels)):
  true_score = row["score"]
  query = queries.loc[row["query-id"]]["text"]
  response = corpus.loc[row["corpus-id"]]["text"]
  res = judge.generate_judgement(query, response)
  res = json.loads(res)
  result = {
    "llm-score": res["score"],
    "human-score": true_score,
    "query-id": row["query-id"],
    "corpus-id": row["corpus-id"]

  }
  results.append(result)

  1%|          | 2/313 [00:45<1:57:11, 22.61s/it]


CancelledError: 

In [ ]:
results_df = pd.DataFrame(results)
from sklearn.metrics import cohen_kappa_score
llm = results_df["llm-score"].to_numpy(int)
human = results_df["human-score"].to_numpy(int)
cohen_kappa_score(llm, human)

0.12605539920011843

In [ ]:
import numpy as np
import metrics


print("NDCG: ", metrics.calc_ndcg(results_df))
print("NRMSE: ", metrics.calc_nrmse(results_df))
print("NMAE: ", metrics.calc_nmae(results_df))
print("kendalltau: ", metrics.calc_kendalltau(results_df))
print("RBO: ", metrics.calc_rbo(results_df))

NDCG:  0.9358348860258707
NRMSE:  0.3840572873934304
NMAE:  0.295
kendalltau:  0.7943678641047063
RBO:  0.9189638548621715


In [ ]:
pd.DataFrame(results_df).to_csv('gemma3:4b_bing_few.tsv', sep='\t', index=False)

In [ ]:
results_df.head(30)

,llm-score,human-score,query-id,corpus-id
0,2,2,0,620
1,2,2,0,621
2,2,2,0,622
3,2,1,0,616
4,2,1,0,617
5,2,1,0,618
6,2,1,0,597
7,2,1,0,598
8,2,1,0,599
9,2,2,1,761
